In [ ]:
import pandas as pd

## Load Data

In [ ]:
# gene_effect = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/gene_effect.csv", index_col=0)
# cell_line_efficacy = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/cell_line_efficacy.csv", index_col=0)
# cell_line_growth_rate = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/cell_line_growth_rate.csv", index_col=0)
# gene_dependency = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/gene_dependency.csv", index_col=0)
# guide_efficacy = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/guide_efficacy.csv", index_col=0)
# common_essentials = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/common_essentials.csv", index_col=0)
omics = pd.read_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_drug_screen/OmicsExpressionProteinCodingGenesTPMLogp1BatchCorrected.csv', index_col=0)
braf_correlates = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/BRAF in Chronos_Combined associations.csv", index_col=0)
egfr_correlates = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/EGFR in Chronos_Combined associations.csv", index_col=0)
kras_correlates = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/KRAS in Chronos_Combined associations.csv", index_col=0)

In [ ]:
# DepMap default for crispr screens
gene_effect_default = pd.read_csv("/home/mike/UZH_USZ/data/DepMap_24Q2/downloads_crispr_screen/CRISPRGeneEffect.csv", index_col=0)

In [ ]:
# Interesting genes
genes_of_interest = ["EGFR", "BRAF"] #, "KRAS"]
genes_of_interest = [gene for gene in gene_effect_default.columns if gene.startswith("EGFR") or gene.startswith("BRAF")]
gene_effect_filtered = gene_effect_default[genes_of_interest]

# We go for braf and egfr, because there are bigger pre-computed associations

## Check Correlations

In [ ]:
gene_effect_filtered.corr()

## Check ATE

In [ ]:
(gene_effect_filtered.iloc[:,1]-gene_effect_filtered.iloc[:,0]).abs().mean()

In [ ]:
gene_effect_filtered.iloc[:,1].hist(bins=100)

In [ ]:
gene_effect_filtered.iloc[:,1].hist(bins=100)

## Omics Data for Cell Lines

In [ ]:
# Filter for cell lines which were screened for both drugs and are contained in cl_id column from filt_outcomes df
omics = omics[omics.index.isin(gene_effect_filtered.index)]

# Filter for cell lines which were screened for both drugs and are contained in cl_id column from filt_outcomes df
filt_outcomes = gene_effect_filtered[gene_effect_filtered.index.isin(omics.index)]

## Get Top Correlates

In [ ]:
braf_correlates

In [ ]:
braf_genes = braf_correlates[braf_correlates["Dataset"] == "Expression Public 24Q2"].index
egfr_genes = egfr_correlates[egfr_correlates["Dataset"] == "Expression Public 24Q2"].index
corr_genes = set(braf_genes.append(egfr_genes))

In [ ]:
selected_genes = [col for col in omics.columns if col.split(" ")[0] in corr_genes]
omics_filtered = omics[selected_genes]

In [ ]:
# Remove all genes with very low variance
omics_filtered = omics_filtered.loc[:, omics_filtered.var() > 0.1]

## Get Outcomes

In [ ]:
filt_outcomes = gene_effect_filtered
filt_outcomes.columns = ["LFC_BRAF", "LFC_EGFR"]

## Combine with outcomes and store

In [ ]:
# Combine omics and outcomes
data = filt_outcomes.join(omics_filtered)
data.dropna(inplace=True)
data.to_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/real/depmap_crispr_screen_2_kos.csv')

In [ ]:
# Normalize data
import numpy as np
data = data - np.min(data, axis=0) / (np.max(data, axis=0) - np.min(data, axis=0))

In [ ]:
data.to_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/real/depmap_crispr_screen_2_kos_norm.csv')

In [ ]:
data_all_features = filt_outcomes.join(omics)

In [ ]:
data_all_features = data_all_features.dropna()

In [ ]:
data_all_features.to_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/real/depmap_crispr_screen_2_genes_all_features.csv')

In [ ]:
import numpy as np
data_all_features_norm = data_all_features - np.min(data_all_features, axis=0) / (np.max(data_all_features, axis=0) - np.min(data_all_features, axis=0))
data_all_features.to_csv('/home/mike/UZH_USZ/data/DepMap_24Q2/real/depmap_crispr_screen_2_genes_all_features_norm.csv')